In [4]:
from multicall import Call
import plotly.express as px
import pandas as pd
from v2_rebalance_dashboard.constants import (
    ROOT_DIR,
    eth_client,
    BALANCER_AURA_DESTINATION_VAULT_ABI, balETH_AUTOPOOL_ETH_ADDRESS, 
    ROOT_PRICE_ORACLE,
)

from v2_rebalance_dashboard.get_state_by_block import (
    safe_normalize_with_bool_success,
    sync_get_raw_state_by_block_one_block,
    sync_safe_get_raw_state_by_block,
    build_blocks_to_use,
)

BalancerAuraDestinationVault = "0xa3956D49106288E5c04E6FBbBad5b68593f0bE3b"
BaseRewardPool4626 = "0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696"  # this is the aura place?
extra_rewards = "0xe5aABCd2D935C3F06a69b981069A92B252E81a99"  # aura
stats = "0xE6850bc48feF4F8e4433d95072177aa95cD2fd9F"

vault_df = pd.read_csv("vaults.csv")
def get_required_addresses_for_balancer_growth_of_a_dollar(destination_vault_address:str):
    BalancerAuraDestinationVault = eth_client.eth.contract(eth_client.toChecksumAddress(destination_vault_address), abi=BALANCER_AURA_DESTINATION_VAULT_ABI)
    auraStaking_adddress = BalancerAuraDestinationVault.functions.auraStaking().call()
    

In [1]:
from multicall import Call
import plotly.express as px

from v2_rebalance_dashboard.constants import (
    ROOT_DIR,
    eth_client,
    BALANCER_AURA_DESTINATION_VAULT_ABI, balETH_AUTOPOOL_ETH_ADDRESS, 
    ROOT_PRICE_ORACLE,
)

from v2_rebalance_dashboard.get_state_by_block import (
    safe_normalize_with_bool_success,
    sync_get_raw_state_by_block_one_block,
    sync_safe_get_raw_state_by_block,
    build_blocks_to_use,
)

#TODO write script to make this for the calls


BalancerAuraDestinationVault = "0xa3956D49106288E5c04E6FBbBad5b68593f0bE3b"
BaseRewardPool4626 = "0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696"  # this is the aura place?
extra_rewards = "0xe5aABCd2D935C3F06a69b981069A92B252E81a99"  # aura
stats = "0xE6850bc48feF4F8e4433d95072177aa95cD2fd9F"
# https://app.aura.finance/#/1/pool/207
# wstETH:ETHx only AURA and BAL ETH tokens, yes solo
destination_vault_contract = eth_client.eth.contract(
    BalancerAuraDestinationVault, abi=BALANCER_AURA_DESTINATION_VAULT_ABI
)
for f in destination_vault_contract.all_functions():
    print(f)

<Function ONE()>
<Function accessController()>
<Function allowance(address,address)>
<Function approve(address,uint256)>
<Function auraBooster()>
<Function auraPoolId()>
<Function auraStaking()>
<Function balanceOf(address)>
<Function balanceOfUnderlyingDebt()>
<Function balancerPool()>
<Function balancerVault()>
<Function baseAsset()>
<Function collectRewards()>
<Function debtValue(uint256)>
<Function decimals()>
<Function decreaseAllowance(address,uint256)>
<Function defaultRewardToken()>
<Function depositUnderlying(uint256)>
<Function exchangeName()>
<Function externalDebtBalance()>
<Function externalQueriedBalance()>
<Function getMarketplaceRewards()>
<Function getPool()>
<Function getRangePricesLP()>
<Function getStats()>
<Function getSystemRegistry()>
<Function getUnderlyerCeilingPrice()>
<Function getUnderlyerFloorPrice()>
<Function getValidatedSafePrice()>
<Function getValidatedSpotPrice()>
<Function increaseAllowance(address,uint256)>
<Function initialize(address,address,addre

In [39]:
# BaseRewardPool4626 = '0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696' # this is the aura place?
# extra_rewards = '0xe5aABCd2D935C3F06a69b981069A92B252E81a99' # aura
# assumes no dilution, at 1 ETH level.
# Our records are real so we dilute the returns, this is hypothetical so without adding a bunch of complexity


def _get_safe_price_from_getRangePricesLP(success, value):
    if success:
        spotPriceInQuote, safePriceInQuote, isSpotSafe = value
        return int(safePriceInQuote) / 1e18


def _build_price_calls(rewardTokens:list[str], rewardTokenSymbols:list[str]) -> list[Call]:
    """Returns a list of calls that get the safe ETH value in ETH for each reward token"""
    price_calls = []
    
    for rewardToken, symbol in zip(rewardTokens, rewardTokenSymbols):
        price_calls.append(
            Call(
                ROOT_PRICE_ORACLE,
                ["getPriceInEth(address)(uint256)", rewardToken],
                [(f'{symbol}_to_ETH', safe_normalize_with_bool_success)],
            )
        )
    return price_calls



def _build_rewardPerToken_calls(mainRewarder:str, extraRewarders:list[str], extraRewardersRewardTokenSymbol:list[str], incentiveStats:str) -> list[Call]:
    """Get the rewardPerToken for each of the rewarder"""
    bal_rewards_call = Call(
        mainRewarder,
        ["rewardPerToken()(uint256)"],
        [("mintedBAL_rewardPerToken", safe_normalize_with_bool_success)],
    )

    aura_minted_from_one_BAL_call =  Call(
        incentiveStats,
        [
            "getPlatformTokenMintAmount(address,uint256)(uint256)",
            AURA,
            int(1e18),
        ],  # imo rounding error, 1.0936 to 1.0696 in a 50 days 1:1 would be 95% as close
        [("AURA_minted_for_one_BAL", safe_normalize_with_bool_success)],
    )
    
    
    rewardPerTokenCalls = []
    for i, (extraRewarder, extraRewardTokenSymbol) in enumerate(zip(extraRewarders, extraRewardersRewardTokenSymbol)):
        rewardPerTokenCall = Call(
        extraRewarder,
        ["rewardPerToken()(uint256)"],
        [(f"{extraRewardTokenSymbol}|| {i} extraRewardPerToken", safe_normalize_with_bool_success)], # using `i` here to avoid duplicates 
    )
        rewardPerTokenCalls.append(rewardPerTokenCall)

    return [bal_rewards_call, aura_minted_from_one_BAL_call, *rewardPerTokenCalls]


def build_growth_of_a_dollar_calls(
    rewardTokens:list[str], rewardTokenSymbols:list[str], mainRewarder:str, extraRewarders:list[str], extraRewardersRewardTokenSymbols: list[str], balancerAuraDestinationVault_address:str, aura_incentive_stats:str
) -> list[Call]:
    price_calls =  _build_price_calls(rewardTokens, rewardTokenSymbols)
    reward_per_token_calls = _build_rewardPerToken_calls(mainRewarder, extraRewarders, extraRewardersRewardTokenSymbols, aura_incentive_stats)
    
    get_safe_lp_token_value_call = Call(
        balancerAuraDestinationVault_address,
        ["getRangePricesLP()((uint256,uint256,bool))"],
        [("safeLPTokenPriceInETH", _get_safe_price_from_getRangePricesLP)],
    )
    
    return [*price_calls, *reward_per_token_calls, get_safe_lp_token_value_call]



BalancerAuraDestinationVault = "0xa3956D49106288E5c04E6FBbBad5b68593f0bE3b"
BaseRewardPool4626_main_rewarder = "0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696"  # this is the aura place?
extra_rewards = "0xe5aABCd2D935C3F06a69b981069A92B252E81a99"  # aura
aura_incentive_stats = "0xE6850bc48feF4F8e4433d95072177aa95cD2fd9F"


AURA = "0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF"
BAL = "0xba100000625a3754423978a60c9317c58a424e3D"

rewardTokens = [AURA, BAL]
rewardTokenSymbols = ['AURA', 'BAL']

growth_of_a_dollar_calls = build_growth_of_a_dollar_calls(
    rewardTokens=rewardTokens, rewardTokenSymbols=rewardTokenSymbols, mainRewarder=BaseRewardPool4626_main_rewarder , extraRewarders=[extra_rewards], extraRewardersRewardTokenSymbols=['AURA'],
    balancerAuraDestinationVault_address=BalancerAuraDestinationVault, aura_incentive_stats = aura_incentive_stats
)

blocks = build_blocks_to_use()
response = sync_get_raw_state_by_block_one_block(growth_of_a_dollar_calls, 20642289)

raw_growth_of_a_dollar_df = sync_safe_get_raw_state_by_block(growth_of_a_dollar_calls, blocks)
growth_of_a_dollar_df = raw_growth_of_a_dollar_df
growth_of_a_dollar_df
# growth_of_a_dollar_df= growth_of_a_dollar_df[growth_of_a_dollar_df.index > '2024, 7, 17'].copy()

,AURA_to_ETH,BAL_to_ETH,mintedBAL_rewardPerToken,AURA_minted_for_one_BAL,AURA|| 0 extraRewardPerToken,safeLPTokenPriceInETH
timestamp,,,,,,
2024-07-08 14:38:47,0.000203,0.000822,15.010115,1.0936,160.278954,1.005999
2024-07-08 20:37:59,0.000199,0.000822,15.057227,1.0936,160.427157,1.005982
2024-07-09 02:34:59,0.000194,0.000822,15.104050,1.0936,160.574453,1.006051
2024-07-09 08:31:59,0.000194,0.000822,15.150873,1.0936,160.721749,1.006949
2024-07-09 14:30:11,0.000201,0.000815,15.197854,1.0936,160.869540,1.006990
...,...,...,...,...,...,...
2024-09-02 14:50:11,0.000199,0.000748,26.747052,1.0680,178.353673,1.010502
2024-09-02 20:46:23,0.000199,0.000748,26.797274,1.0680,178.412906,1.010502
2024-09-03 02:42:59,0.000195,0.000763,26.847552,1.0680,178.472205,1.010423


In [50]:
def transform_groth_of_a_dollar_df_to_growth_of_1_ETH_with_no_entrance_costs(raw_growth_of_a_dollar_df:pd.DataFrame, mock_date_deployed:str = '2024, 7, 17'):
    growth_of_a_dollar_df =  raw_growth_of_a_dollar_df[raw_growth_of_a_dollar_df.index > mock_date_deployed].copy()
    growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] = 1 / growth_of_a_dollar_df['safeLPTokenPriceInETH'].values[0] # constant
    growth_of_a_dollar_df['safe_value_of_lp_tokens'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * growth_of_a_dollar_df['safeLPTokenPriceInETH']

    _add_BAL_and_AURA_minted_eth_value(growth_of_a_dollar_df)
    _add_extra_rewarders_incentive_eth_value(growth_of_a_dollar_df)
    growth_of_a_dollar_df['total_cumulative_incentive_tokens_current_eth_value'] = growth_of_a_dollar_df['extra_rewarder_cumulative_incentive_eth_value'] + growth_of_a_dollar_df['main_rewarder_cumulative_incentive_eth_value']
    growth_of_a_dollar_df['growth_of_1_ETH'] = growth_of_a_dollar_df['safe_value_of_lp_tokens'] + growth_of_a_dollar_df['total_cumulative_incentive_tokens_current_eth_value']
    return growth_of_a_dollar_df



def _add_BAL_and_AURA_minted_eth_value(growth_of_a_dollar_df:pd.DataFrame):
    starting_mintedBAL_rewardPerToken  = growth_of_a_dollar_df['mintedBAL_rewardPerToken'].values[0]
    growth_of_a_dollar_df['BAL_minted_since_start_date'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * (growth_of_a_dollar_df['mintedBAL_rewardPerToken'] - starting_mintedBAL_rewardPerToken)
    growth_of_a_dollar_df['AURA_minted_since_start_date'] = growth_of_a_dollar_df['BAL_minted_since_start_date'] * growth_of_a_dollar_df['AURA_minted_for_one_BAL']
    growth_of_a_dollar_df['main_rewarder_cumulative_incentive_eth_value'] = (growth_of_a_dollar_df['BAL_minted_since_start_date'] * growth_of_a_dollar_df['BAL_to_ETH']) + (growth_of_a_dollar_df['AURA_minted_since_start_date'] * growth_of_a_dollar_df['AURA_to_ETH'])
    

def _add_extra_rewarders_incentive_eth_value(growth_of_a_dollar_df:pd.DataFrame):
    
    extra_reward_per_token_cols = [c for c in growth_of_a_dollar_df.columns if 'extraRewardPerToken' in c] # format like AURA|| 0 extraRewardPerToken
    extra_reward_to_eth_cols = [f"{c.split('||')[0]}_to_ETH" for c in extra_reward_per_token_cols]
    print(f'{extra_reward_per_token_cols=} {extra_reward_to_eth_cols=}')
    growth_of_a_dollar_df['extra_rewarder_cumulative_incentive_eth_value'] = 0.0
    for extra_rewardPerToken_col, ETH_price_col in zip(extra_reward_per_token_cols, extra_reward_to_eth_cols):
        starting_rewardPerToken = growth_of_a_dollar_df[extra_rewardPerToken_col].values[0]
        extra_rewards_minted_since_start = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * (growth_of_a_dollar_df[extra_rewardPerToken_col] - starting_rewardPerToken)
        growth_of_a_dollar_df['extra_rewarder_cumulative_incentive_eth_value'] += extra_rewards_minted_since_start * growth_of_a_dollar_df[ETH_price_col]
        
growth_of_a_dollar_df = transform_groth_of_a_dollar_df_to_growth_of_1_ETH_with_no_entrance_costs(raw_growth_of_a_dollar_df)
px.line(growth_of_a_dollar_df['growth_of_1_ETH'])

extra_reward_per_token_cols=['AURA|| 0 extraRewardPerToken'] extra_reward_to_eth_cols=['AURA_to_ETH']


In [49]:
growth_of_a_dollar_df['safe_value_of_lp_tokens']

timestamp
2024-07-17 00:53:47    1.000000
2024-07-17 06:50:11    0.999990
2024-07-17 12:46:35    1.000138
2024-07-17 18:43:35    1.000117
2024-07-18 00:40:11    1.000117
                         ...   
2024-09-02 14:50:11    1.003516
2024-09-02 20:46:23    1.003516
2024-09-03 02:42:59    1.003438
2024-09-03 08:40:23    1.003431
2024-09-03 14:36:47    1.003474
Name: safe_value_of_lp_tokens, Length: 197, dtype: float64

In [34]:
raw_growth_of_a_dollar_df.columns

Index(['AURA', 'BAL', 'mintedBAL_rewardPerToken', 'AURA_minted_for_one_BAL',
       'AURA 0 rewardPerToken', 'safeLPTokenPriceInETH'],
      dtype='object')

In [4]:
growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] = 1 / growth_of_a_dollar_df['safeLPTokenPriceInETH'].values[0]
growth_of_a_dollar_df['safe_value_of_lp_tokens'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * growth_of_a_dollar_df['safeLPTokenPriceInETH']

# need to add some assumptions in here about the gas costs of entering, (that we paid) ie 200 usd
# and the slippage cost to enter? using our values lets say 
starting_bal_reward_per_token  = growth_of_a_dollar_df['BAL_reward_per_token'].values[0]
starting_stash_aura_reward_per_token = growth_of_a_dollar_df['stash_aura_reward_per_token'].values[0]

growth_of_a_dollar_df['bal_earned_since_deposit'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * (growth_of_a_dollar_df['BAL_reward_per_token'] - starting_bal_reward_per_token)
growth_of_a_dollar_df['aura_minted_since_deposit'] = growth_of_a_dollar_df['bal_earned_since_deposit'] * growth_of_a_dollar_df['AURA_minted_for_one_BAL'] 

growth_of_a_dollar_df['stash_aura_earned_since_deposit'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * (growth_of_a_dollar_df['stash_aura_reward_per_token'] - starting_stash_aura_reward_per_token)

growth_of_a_dollar_df['current_eth_value_of_incentive_tokens_earned'] = (growth_of_a_dollar_df['bal_earned_since_deposit'] * growth_of_a_dollar_df['BAL_to_ETH']) + (growth_of_a_dollar_df['aura_minted_since_deposit'] * growth_of_a_dollar_df['AURA_to_ETH']) + (growth_of_a_dollar_df['stash_aura_earned_since_deposit'] * growth_of_a_dollar_df['AURA_to_ETH']) 
growth_of_a_dollar_df['wstETH_ETHx_balancer_value_of_one_eth'] =growth_of_a_dollar_df['safe_value_of_lp_tokens'] + growth_of_a_dollar_df['current_eth_value_of_incentive_tokens_earned']
px.line(growth_of_a_dollar_df[['wstETH_ETHx_balancer_value_of_one_eth', 'our_nav_per_share']], title='Nav Per Share vs Balancer wstETH/ETHx')

In [10]:
# account for costs

growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] = (1 / growth_of_a_dollar_df['safeLPTokenPriceInETH'].values[0]) * (1 - .0035) # assume 35 bips to enter and 10bips of gas cost
growth_of_a_dollar_df['safe_value_of_lp_tokens'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * growth_of_a_dollar_df['safeLPTokenPriceInETH']

# need to add some assumptions in here about the gas costs of entering, (that we paid) ie 200 usd
# and the slippage cost to enter? using our values lets say 
starting_bal_reward_per_token  = growth_of_a_dollar_df['BAL_reward_per_token'].values[0]
starting_stash_aura_reward_per_token = growth_of_a_dollar_df['stash_aura_reward_per_token'].values[0]

growth_of_a_dollar_df['bal_earned_since_deposit'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * (growth_of_a_dollar_df['BAL_reward_per_token'] - starting_bal_reward_per_token)
growth_of_a_dollar_df['aura_minted_since_deposit'] = growth_of_a_dollar_df['bal_earned_since_deposit'] * growth_of_a_dollar_df['AURA_minted_for_one_BAL'] 

growth_of_a_dollar_df['stash_aura_earned_since_deposit'] = growth_of_a_dollar_df['starting_quantity_of_lp_tokens'] * (growth_of_a_dollar_df['stash_aura_reward_per_token'] - starting_stash_aura_reward_per_token)

growth_of_a_dollar_df['current_eth_value_of_incentive_tokens_earned'] =  (growth_of_a_dollar_df['bal_earned_since_deposit'] * growth_of_a_dollar_df['BAL_to_ETH']) + (growth_of_a_dollar_df['aura_minted_since_deposit'] * growth_of_a_dollar_df['AURA_to_ETH']) + (growth_of_a_dollar_df['stash_aura_earned_since_deposit'] * growth_of_a_dollar_df['AURA_to_ETH']) 

growth_of_a_dollar_df['current_eth_value_of_incentive_tokens_earned'] =  growth_of_a_dollar_df['current_eth_value_of_incentive_tokens_earned'] # assume 35 ps of cost to sell reward tokens?
growth_of_a_dollar_df['wstETH_ETHx_balancer_value_of_one_eth'] =growth_of_a_dollar_df['safe_value_of_lp_tokens'] + growth_of_a_dollar_df['current_eth_value_of_incentive_tokens_earned']
px.line(growth_of_a_dollar_df[['wstETH_ETHx_balancer_value_of_one_eth', 'our_nav_per_share']], title='Nav Per Share vs Balancer wstETH/ETHx if it cost 35bps to enter')

In [6]:
(20.40 / 20.42 ) - 1 # 10 bips
(59.79  / 60) - 1 # 35 bips +
((35 * 60 ) + (10 * 20)) / 80
# avg of 29 bips of slippage give them 

28.75

In [ ]:
# # BaseRewardPool4626 = '0x571a20C14a7c3Ac6d30Ee7D1925940bb0C027696' # this is the aura place?
# # extra_rewards = '0xe5aABCd2D935C3F06a69b981069A92B252E81a99' # aura
# # assumes no dilution, at 1 ETH level.
# # Our records are real so we dilute the returns, this is hypothetical so without adding a bunch of complexity


# def _get_safe_price_from_getRangePricesLP(success, value):
#     if success:
#         spotPriceInQuote, safePriceInQuote, isSpotSafe = value
#         return int(safePriceInQuote) / 1e18


# def build_reward_per_token_stored_calls():
#     AURA = "0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF"
#     BAL = "0xba100000625a3754423978a60c9317c58a424e3D"
#     bal_rewards_call = Call(
#         BaseRewardPool4626,
#         ["rewardPerToken()(uint256)"],
#         [("BAL_reward_per_token", safe_normalize_with_bool_success)],
#     )

#     aura_stash_rewards_call = Call(
#         extra_rewards,
#         ["rewardPerToken()(uint256)"],
#         [("stash_aura_reward_per_token", safe_normalize_with_bool_success)],
#     )

#     aura_minted_from_one_BAL_call = Call(
#         stats,
#         [
#             "getPlatformTokenMintAmount(address,uint256)(uint256)",
#             AURA,
#             int(1e18),
#         ],  # imo rounding error, 1.0936 to 1.0696 in a 50 days 1:1 would be 95% as close
#         [("AURA_minted_for_one_BAL", safe_normalize_with_bool_success)],
#     )

#     get_safe_lp_token_value_call = Call(
#         BalancerAuraDestinationVault,
#         ["getRangePricesLP()((uint256,uint256,bool))"],
#         [("safeLPTokenPriceInETH", _get_safe_price_from_getRangePricesLP)],
#     )
#     price_calls = []
#     for rewardToken, name in zip([AURA, BAL], ["AURA_to_ETH", "BAL_to_ETH"]):
#         price_calls.append(
#             Call(
#                 ROOT_PRICE_ORACLE,
#                 ["getPriceInEth(address)(uint256)", rewardToken],
#                 [(name, safe_normalize_with_bool_success)],
#             )
#         )
        
#     our_nav_per_share = Call(
#         balETH_AUTOPOOL_ETH_ADDRESS,
#         ["convertToAssets(uint256)(uint256)", int(1e18)],
#         [('our_nav_per_share', safe_normalize_with_bool_success)],
#     )

#     return [bal_rewards_call, aura_minted_from_one_BAL_call, aura_stash_rewards_call, get_safe_lp_token_value_call, *price_calls, our_nav_per_share]


# calls = build_reward_per_token_stored_calls()
# blocks = build_blocks_to_use()
# response = sync_get_raw_state_by_block_one_block(calls, 20642289)
# growth_of_a_dollar_df = sync_safe_get_raw_state_by_block(calls, blocks)
# growth_of_a_dollar_df= growth_of_a_dollar_df[growth_of_a_dollar_df.index > '2024, 7, 17'].copy()